<a href="https://colab.research.google.com/github/AlexKressner/Industrielles_Management/blob/main/ErweiterungenNetworkDesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Design

In [118]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [119]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Daten aus csv-Datei laden



In [120]:
! git clone https://github.com/AlexKressner/Industrielles_Management

fatal: destination path 'Industrielles_Management' already exists and is not an empty directory.


In [121]:
# Daten laden
import pandas as pd

In [122]:
folder = "Industrielles_Management/Daten/NetworkDesign"

In [123]:
path = f"{folder}/NetworkDesign_Invest_Kapa.csv"

In [124]:
kapa=pd.read_csv(path, sep=";")

In [125]:
kapa.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [126]:
path = f"{folder}/NetworkDesign_Produktionskosten.csv"

In [127]:
produktionskosten=pd.read_csv(path, sep=";")

In [128]:
produktionskosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [129]:
path = f"{folder}/NetworkDesign_Nachfrage.csv"

In [130]:
nachfrage=pd.read_csv(path, sep=";")

In [131]:
nachfrage.head()

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


## Indexmengen

In [132]:
I = produktionskosten["Produktionsstandort"].unique().tolist() # Menge der Produktionsstandorte

In [133]:
I

['Charleston', 'Curitiba', 'Hamburg', 'Quanzhou', 'East London']

# Erweiterung 2: Neuer Markt Arabische Halbinsel (AH)

In [134]:
J = nachfrage["Markt"].tolist() #Menge der Märkte

In [135]:
J.append("AH")

In [136]:
J

['USA', 'Südamerika', 'Europa', 'Asien', 'Afrika', 'AH']

In [137]:
A = kapa["Ausbaustufe"].unique().tolist() # Menge der Ausbaustufen an einem Standort

In [138]:
A

['klein', 'groß']

## Entscheidungsvariablen

In [139]:
infinity = solver.infinity()

In [140]:
# Definition der Entscheidungsvariablen
# x sind die Flussvariablen, d.h. Herstell- und Transportmenge von i nach j
x={}
for i in I:
  for j in J:
    x[i,j] = solver.IntVar(0, infinity, f"{i},{j}")

In [141]:
# Definition der Entscheidungsvariablen
# y sind die Strukturvariablen, d.h. ob und mit welchen Kapazitäten ein Standort ausgebaut wird
y={}
for i in I:
  for a in A:
    y[i,a] = solver.BoolVar(f"{i},{a}")

In [142]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 40


## Parameter

In [143]:
# Herstell- und Transportkosten
produktionskosten.set_index(["Produktionsstandort","Markt"], inplace=True)

In [144]:
c = produktionskosten.to_dict("dict")["Produktionskosten"]

# Erweiterung 2: Neuer Markt Arabische Halbinsel (AH)

In [145]:
c["Charleston","AH"] = 125
c["Curitiba","AH"] = 100
c["Hamburg","AH"] = 100
c["Quanzhou","AH"] = 75
c["East London","AH"] = 85

In [146]:
c

{('Charleston', 'USA'): 81,
 ('Charleston', 'Südamerika'): 92,
 ('Charleston', 'Europa'): 101,
 ('Charleston', 'Asien'): 130,
 ('Charleston', 'Afrika'): 115,
 ('Curitiba', 'USA'): 117,
 ('Curitiba', 'Südamerika'): 77,
 ('Curitiba', 'Europa'): 108,
 ('Curitiba', 'Asien'): 98,
 ('Curitiba', 'Afrika'): 100,
 ('Hamburg', 'USA'): 102,
 ('Hamburg', 'Südamerika'): 105,
 ('Hamburg', 'Europa'): 95,
 ('Hamburg', 'Asien'): 119,
 ('Hamburg', 'Afrika'): 111,
 ('Quanzhou', 'USA'): 115,
 ('Quanzhou', 'Südamerika'): 125,
 ('Quanzhou', 'Europa'): 90,
 ('Quanzhou', 'Asien'): 59,
 ('Quanzhou', 'Afrika'): 74,
 ('East London', 'USA'): 142,
 ('East London', 'Südamerika'): 100,
 ('East London', 'Europa'): 103,
 ('East London', 'Asien'): 105,
 ('East London', 'Afrika'): 71,
 ('Charleston', 'AH'): 125,
 ('Curitiba', 'AH'): 100,
 ('Hamburg', 'AH'): 100,
 ('Quanzhou', 'AH'): 75,
 ('East London', 'AH'): 85}

In [147]:
# Nachfrage aus den Märtken

In [148]:
nachfrage.set_index(["Markt"], inplace=True)

In [149]:
d = nachfrage.to_dict("dict")["Nachfragemenge"]

# Erweiterung 2: Neuer Markt Arabische Halbinsel (AH)

In [150]:
d["AH"] = 5

In [151]:
d

{'USA': 12, 'Südamerika': 8, 'Europa': 14, 'Asien': 16, 'Afrika': 7, 'AH': 5}

In [152]:
# Kapazitäten der Ausbaustufen und Investitionskosten
kapazitaeten = kapa.set_index("Ausbaustufe")

In [153]:
cap = kapazitaeten.to_dict("dict")["Kapazitäten"]

In [154]:
cap

{'klein': 10, 'groß': 20}

In [155]:
f = kapa.set_index(["Produktionsstandort","Ausbaustufe"]).to_dict("dict")["Investitionen"]

# Erweiterung 1: Subventionen für den Bau einer Fabrik in den USA von 30%

In [156]:
f["Charleston","klein"] = f["Charleston","klein"] * 0.7

In [157]:
f["Charleston","groß"] = f["Charleston","groß"] * 0.7

## Zielfunktion

min $K=\sum_{i,j} c_{ij}* X_{ij} + \sum_{i,a} fc_{ia}* Y_{ia}$

In [158]:
# Maximierung der gesamten Deckungsbeiträge
solver.Minimize(sum(c[i,j]*x[i,j] for i in I for j in J) + sum(f[i,a]*y[i,a] for i in I for a in A))

## Nebenbedingungen

$\sum_i X_{ij} \ge d_j, ∀j$  

In [159]:
# Nachfrage in den Märkten erfüllen
for j in J:
  solver.Add(sum(x[i,j] for i in I)>=d[j])

$\sum_j X_{ij} \le \sum_a cap_{ia} * Y_{ia}, ∀i$

In [160]:
# Kapazitäten aufbauen und einhalten
for i in I:
  solver.Add(sum(x[i,j] for j in J)<= sum(cap[a]*y[i,a] for a in A))

## Berechnung Lösung

In [161]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Kosten) =', solver.Objective().Value())
    print("\n")
    print("Produktions-Markt-Zuteilung")
    for j in J:
      print(f"Markt:{j}")
      for i in I:
        if x[i,j].solution_value()>0:
          print(f'{i,j} =', round(x[i,j].solution_value()))
    print("\n")
    print("Invest Standorte")
    for i in I:
      for a in A:
        if y[i,a].solution_value()>0:
          print(f'{i,a} =', round(y[i,a].solution_value()))
    print("\n")
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Kosten) = 27383.999999999996


Produktions-Markt-Zuteilung
Markt:USA
('Charleston', 'USA') = 12
Markt:Südamerika
('Charleston', 'Südamerika') = 8
Markt:Europa
('Quanzhou', 'Europa') = 14
Markt:Asien
('Quanzhou', 'Asien') = 16
Markt:Afrika
('East London', 'Afrika') = 7
Markt:AH
('East London', 'AH') = 5


Invest Standorte
('Charleston', 'groß') = 1
('Quanzhou', 'klein') = 1
('Quanzhou', 'groß') = 1
('East London', 'groß') = 1


